##  RC2rot RC and q calculation (3 direction (psi) data) 

mathod A (RC2rot): RC -> (trim) -> rotation 

psi = 0, 120, -12 or 0, 90, -90

(1) Verify the angle file.

(2) Rocking curve (RC) 

- HW fitting angle = 0, 120(p120), -120(m120)

- gauss fitting angle = 0, 120(p120), -120(m120)

Note: The fitting time depends on the capability of your PC.

(3) Image trimming

(4) Prepare the rotating data using image_editor

(5) Calculate q

(6) re-plot and re-analysis 

In [ ]:
# from pathlib import Path

# import matplotlib.pyplot as plt
# import mpl_toolkits.axes_grid1
# from mpl_toolkits.mplot3d import Axes3D

# import numpy as np
# import pandas as pd
# from PIL import Image

In [ ]:
from qfit import make_angle_file as maf
from qfit import fit_q
from qfit import re_analysis as rean
from qfit import re_plot as replt
from qfit import multiplot as mlplt
from qfit import image_treat as imt
from qfit import file_folder_trans as fft

# Detector size
# Set according to the experimental setup.
NX=2368
NY=2240

In [ ]:
import warnings
warnings.simplefilter('ignore')
# warnings.resetwarnings()
# warnings.simplefilter('ignore', FutureWarning)
%load_ext autoreload
%autoreload 2

In [ ]:
# Set target holder path
# g_0 = r"D:\XrayDataset_for_paper\GaN_n1_4_0"
# g_p120 = r"D:\XrayDataset_for_paper\GaN_n1_4_120"
# g_m120 = r"D:\XrayDataset_for_paper\GaN_n1_4_m120"

g_0 =r''
g_p120 = r''
g_m120 = r''

### Check angle file

In [ ]:
# Check if it contains tif data, angle.text, and dark tif data.

# phi=0
_ = maf.check_holder(data_path=g_0)

# phi=p120
_ = maf.check_holder(data_path=g_p120)

# phi=m120
_ = maf.check_holder(data_path=g_m120)

# If the angle file does not exist, go back to creating the angle file. -> temp_anglefile.ipynb

In [ ]:
# If the angle file exists, check the contents of the angle file 
maf.anglefile_info(data_path=g_0)
print('-'*5)
maf.anglefile_info(data_path=g_p120)
print('-'*5)
maf.anglefile_info(data_path=g_m120)

### RC calculation

In [ ]:
# RC hw fitting

# core: Use multi-process PC cores. Attention memory size.

# filter: Filter parameters are set to exclude noise level signals from the analysis. 
# It is only analyzed if the difference between the maximum and minimum RC intensities is greater than the filter value. 
# If it is not included in the analysis, enter the value of Nan as the analysis value.

# method: hw (full width half maxima) 

cores = 4

t_file = g_0
t_folder,_ = fit_q.fit_analysis(target_file=t_file, filter=30, method='hw', comment='0data', core=cores)
print('')

c_file = g_p120
c_folder,_ = fit_q.fit_analysis(target_file=c_file, filter=30, method='hw', comment='p120data', core=cores)
print('')

a_file = g_m120
a_folder,_ = fit_q.fit_analysis(target_file=a_file, filter=30,  method='hw', comment='m120data', core=cores)
print('')

# RC gauss fitting

# pmax: In addition to the filter condition, 
# if the condition with low distribution intensity (maximum intensity> median intensity + PMAX) is not satisfied,
# it is excluded from the analysis. If it is not analyzed, or if the fitting fails, the value of Nan is entered.

# method: gaussian 

# t_file = g_0
# t_folder, _ = fit_q.fit_analysis(target_file=t_file, filter=30, pmax=30, method='gaussian', comment='0data', core=cores)
# print('')

# c_file = g_p120
# c_folder,_ = fit_q.fit_analysis(target_file=c_file, filter=30, pmax=30, method='gaussian', comment='p120data', core=cores)
# print('')

# a_file = g_m120
# a_folder,_ = fit_q.fit_analysis(target_file=a_file, filter=30, pmax=30, method='gaussian', comment='m120data', core=cores)
# print('')



In [ ]:
# Check results

rc0 = rean.load_rc_tif(t_folder)
mlplt.rc_12plots(rc0, title='$\psi$=0')

rcp120 = rean.load_rc_tif(c_folder)
mlplt.rc_12plots(rcp120, title='$\psi$=p120')

rcm120 = rean.load_rc_tif(a_folder)
mlplt.rc_12plots(rcm120, title='$\psi$=m120')

### Trimming

<img src="./docs/figs/triming1.JPG" style="zoom:25%;">
<img src="./docs/figs/triming2.JPG" style="zoom:25%;">

- Wait a few seconds and a new GUI window will appear.
- Area select: Right-click 
- Decide and quit: Press 'ESC' key

In [ ]:
# psi=0
# 2 inch: wh=1100, 4 inch: wh=2150, M:wh=500
# Wait a few seconds and a new GUI window will appear.
wh4 = 2150
wh2 = 1100
whm = 500
wh_set = wh4
tr_t_folder = imt.gui2trim(t_folder, wh=wh_set, NX=NX, NY=NY, time_out=120)

In [ ]:
# psi=p120
tr_c_folder = imt.gui2trim(c_folder, wh=wh_set, NX=NX, NY=NY, time_out=120)

In [ ]:
# psi=m120
tr_a_folder = imt.gui2trim(a_folder, wh=wh_set, NX=NX, NY=NY, time_out=120)

In [ ]:
# Check results
rc0 = rean.load_rc_tif(tr_t_folder)
mlplt.rc_12plots(rc0, title='$\psi$=0')
# mlplt.rc_3plots(rc0, title='$\phi$=0', save=False, dpi=1800)

rcp120 = rean.load_rc_tif(tr_c_folder)
mlplt.rc_12plots(rcp120, title='$\psi$=p120')

rcm120 = rean.load_rc_tif(tr_a_folder)
mlplt.rc_12plots(rcm120,  title='$\psi$=m120')


### Rotating image using Image_editor

<img src="./docs/figs_affin/img_edit.PNG" style="zoom:50%;">

(1) Upper left window: Select data folder (psi=0)

(2) Set the reading step (1: all files, default 16)

(3) Middle left window: Select data folder (e.g. psi=120)

(4) Set the reading step (1: All files, default 16)

(5) If you need to flip the x-direction, set the scale original x to -1.

(6) Change the x,y, rotation, and scale parameters.

(7) Export from the File menu.

(8) Quit from the File menu.



In [ ]:
#  Wait a few seconds (10s - 60s) and a new GUI window will appear.
outfolderlist,_ = imt.img_editor_process()
print(outfolderlist)

# out put folder name :rot_ + 'load folder name' ex: rot_tr_hw_220208_132247
# if missing outfolderlist, input the folder name.
# outfolderlist = ['','']

In [ ]:
# Check results
rc0 = rean.load_rc_tif(outfolderlist[0])
mlplt.rc_12plots(rc0, title='$\psi$=0')

rcp120 = rean.load_rc_tif(outfolderlist[1])
mlplt.rc_12plots(rcp120, title='$\psi$=p120')

rcm120 = rean.load_rc_tif(outfolderlist[2])
mlplt.rc_12plots(rcm120, title='$\psi$=m120')


### h Calculation (Q calculation)

In [ ]:
# Set the data folder after rotation.

# fr0=r'C:\Users\USER\Desktop\XRTD\xrdt\tr_hw_211226_215327'
# frp120=r'C:\Users\USER\Desktop\XRTD\xrdt\tr_hw_211226_215523'

fr0 = outfolderlist[0]
frp120 = outfolderlist[1]
frm120 = outfolderlist[2]

print(fr0,frp120,frm120)

In [ ]:
# convert tif to npy, because the program that calculates Q accepts only npy format.

r0 = fft.conv_tif2npy(file_dir=fr0)
rp120 = fft.conv_tif2npy(file_dir=frp120)
rm120 = fft.conv_tif2npy(file_dir=frm120)

# fft.fd_tif2fd_npy(file_dir=fr0)
# fft.fd_tif2fd_npy(file_dir=frp120)
# fft.fd_tif2fd_npy(file_dir=frm120)

# r0=fft.folder_file_list(fr0,look_for='npy')
# rp120=fft.folder_file_list(frp120,look_for='npy')
# rm120=fft.folder_file_list(frm120,look_for='npy')

print(f'psi=0')
print(r0)

print(f'psi=p120')
print(rp120)

print(f'psi=m120')
print(rm120)

In [ ]:
# The '_c.npy' file is used to calculate q.

target_t_ = r0[fft.search_list_with_wildcard(r0)[0]]
target_c_ = rp120[fft.search_list_with_wildcard(rp120)[0]]

# set output folder name
# If this holder name already exists, use another name or delete the holder.
out_file_ = '2R_0p120'
# phi=0
set_angle_t = 0 
# phi=120
set_angle_c = 120

# If it is trimmed, change NX, NY.
# original NX=2368, NY=2240, 
out_q_folder_0p = fit_q.q_analysis_2R(target_t=target_t_, target_c=target_c_, out_file=out_file_, 
                                    angle_t=set_angle_t, angle_c=set_angle_c, 
                                    q=6.258, NX=wh_set, NY=wh_set)

In [ ]:
# Check results
q_0p = rean.load_q_tif(out_q_folder_0p)
mlplt.qcp_12plots(q_0p, title='q (0,120)', step=100)

In [ ]:
target_t_ = r0[fft.search_list_with_wildcard(r0)[0]]
target_c_ = rm120[fft.search_list_with_wildcard(rm120)[0]]

# set output folder name
out_file_ = '2R_0m120'
# phi=0
set_angle_t = 0 
# phi=120
set_angle_c = -120

# If it is trimmed, change NX, NY.
out_q_folder_0m = fit_q.q_analysis_2R(target_t=target_t_, target_c=target_c_, out_file=out_file_, 
                                    angle_t=set_angle_t, angle_c=set_angle_c, 
                                    q=6.258, NX=wh_set, NY=wh_set)

In [ ]:
# Check results
q_0m = rean.load_q_tif(out_q_folder_0m)
mlplt.qcp_12plots(q_0m, title='q (0,m120)', step=100)

In [ ]:
target_t_ = rp120[fft.search_list_with_wildcard(rp120)[0]]
target_c_ = rm120[fft.search_list_with_wildcard(rm120)[0]]

# set output folder name
out_file_ = '2R_pm120'
# phi=0
set_angle_t = 120 
# phi=120
set_angle_c = -120

# If it is trimmed, change NX, NY.
out_q_folder_pm = fit_q.q_analysis_2R(target_t=target_t_, target_c=target_c_, out_file=out_file_, 
                                    angle_t=set_angle_t, angle_c=set_angle_c, 
                                    q=6.258, NX=wh_set, NY=wh_set)

In [ ]:
# Check results
q_pm = rean.load_q_tif(out_q_folder_pm)
mlplt.qcp_12plots(q_pm, title='q (p120,m120)', step=100)

In [ ]:
# average q
ave_q = rean.average_3q(q_0p, q_0m, q_pm)
mlplt.qcp_12plots(ave_q, title='ave q', step=100)
mlplt.qp_4plots(ave_q, title='q polar')
mlplt.qp3_4plots(ave_q, title='q polar',step=100)
mlplt.qp3_3plots(ave_q, title='q polar analysis', step=150)

In [ ]:
# Other Plot
# out_q_folder_0p='2R_0p120'
# out_q_folder_0m='2R_0m120'
# out_q_folder_pm='2R_pm120'

# q_0p = rean.load_q_tif(out_q_folder_0p)
mlplt.qcp_12plots(q_0p, title='q (0,120)', step=100)
mlplt.qp_4plots(q_0p, title='q polar')
mlplt.qp3_3plots(q_0p, title='q polar analysis', step=150)


In [ ]:
# q_0m = rean.load_q_tif(out_q_folder_0m)
mlplt.qcp_12plots(q_0m, title='q (0,m120)', step=100)

In [ ]:
# q_pm = rean.load_q_tif(out_q_folder_pm)
mlplt.qcp_12plots(q_pm, title='q (p120,m120)', step=100)